In [27]:
import glob2
import pickle
import os
import ast
import json

In [34]:
def convert_annotations(annotation, string = True):
    if 'misinformation' in annotation.keys() and 'true'in annotation.keys():
        if int(annotation['misinformation'])==0 and int(annotation['true'])==0:
            if string:
                label = "unverified"
            else:
                label = 2
        elif int(annotation['misinformation'])==0 and int(annotation['true'])==1 :
            if string:
                label = "true"
            else:
                label = 1
        elif int(annotation['misinformation'])==1 and int(annotation['true'])==0 :
            if string:
                label = "false"
            else:
                label = 0
        elif int(annotation['misinformation'])==1 and int(annotation['true'])==1:
            print ("OMG! They both are 1!")
            print(annotation['misinformation'])
            print(annotation['true'])
            label = None
            
    elif 'misinformation' in annotation.keys() and 'true' not in annotation.keys():
        # all instances have misinfo label but don't have true label
        if int(annotation['misinformation'])==0:
            if string:
                label = "unverified"
            else:
                label = 2
        elif int(annotation['misinformation'])==1:
            if string:
                label = "false"
            else:
                label = 0
                
    elif 'true' in annotation.keys() and 'misinformation' not in annotation.keys():
        print ('Has true not misinformation')
        label = None
    else:
        print('No annotations')
        label = None
           
    return label

In [40]:
event_labels=['charliehebdo','sydneysiege','germanwings-crash','ferguson','ottawashooting']
root_path='rumour prediction/data/all-rnr-annotated-threads/'

# total reaction count
# reactions=[]
# for e in event_labels:
#     path=root_path + '{}/rumours'.format(e)
#     dirs=os.listdir(path)
#     for d in dirs:
#       if(d=='.DS_Store'):
#         continue
#       reactions.extend(glob2.glob(path + '/{}/reactions/*.json'.format(d) ))  
# reactions=[r.split('/')[-1].replace('.json','') for r in reactions]  

veracity_enum = { 
  'nonrumour':0,
  'rumour':1,
  'unclear':2
}

labels={'charliehebdo':veracity_dict,'sydneysiege':veracity_dict,\
        'germanwings-crash':veracity_dict,'ferguson':veracity_dict,'ottawashooting':veracity_dict}
root_dir='rumour prediction/data/all-rnr-annotated-threads'
for e in event_labels:
    veracity_dict={'true':0,'false':0,'unverified':0}
    path=root_dir  + '/{}'.format(e) + '/rumours/'
    dirs=os.listdir(path)
    for d in dirs:
      d=d.replace('._','')
      if(d=='.DS_Store'):
        continue
      with open(path + '/' + d + '/annotation.json','r') as f:
        veracity=json.load(f)
        if(veracity is not None):
#           labels.append(veracity['is_rumour'])
#           veracity_dict.update({veracity_enum[veracity['is_rumour']]:d})
            veracity_dict[convert_annotations(veracity)]+=1
        else:
          print('annotation none for :{}'.format(d))  
    labels[e]=veracity_dict    

In [41]:
labels

{'charliehebdo': {'true': 193, 'false': 116, 'unverified': 149},
 'sydneysiege': {'true': 382, 'false': 86, 'unverified': 54},
 'germanwings-crash': {'true': 94, 'false': 111, 'unverified': 33},
 'ferguson': {'true': 10, 'false': 8, 'unverified': 266},
 'ottawashooting': {'true': 329, 'false': 72, 'unverified': 69}}

In [21]:
stance_enum={
    'agreed':0,
    'comment':1,
    'appeal-for-more-information':2,
    'disagreed':3
}

In [164]:
root_path='rumour prediction/data/all-rnr-annotated-threads/'
replies={'charliehebdo':[],'sydneysiege':[],\
        'germanwings-crash':[],'ferguson':[],'ottawashooting':[]}
sources={'charliehebdo':[],'sydneysiege':[],\
        'germanwings-crash':[],'ferguson':[],'ottawashooting':[]}
for e in event_labels:
    veracity_dict={'true':0,'false':0,'unverified':0}
    path=root_path  + e + '/rumours'
    dirs=os.listdir(path)
    for d in dirs:
      d=d.replace('._','')
      if(d=='.DS_Store'):
        continue 
#       print(path + '/{}/reactions/*.json'.format(d))
#       react=glob2.glob((path + '/{}/reactions/*.json'.format(d)))
#       react=[(r.split('/')[-1]).replace('.json','') for r in react]
      with open(path + '/{}/structure.json'.format(d),'r')as f:
        react=f.read()
      react=react.replace('{','').replace('}','').replace('[]','').replace(':',' ')\
                .replace('\"','').replace(',',' ').split()
      replies[e].extend(react[1:])
#       sources[e].append(d)

In [165]:
import copy

stance={}
with open('rumour prediction/data/all-rnr-annotated-threads/stance-annotations.json','r') as f:
  lines=f.readlines()
for line in lines:
  if('#' in line or 'putinmissing' in line \
     or 'prince-toronto' in line or 'ebola-essien' in line):
    continue
  annot=ast.literal_eval(line)
  if('responsetype-vs-source' in annot.keys()):
    stance[annot['tweetid']]=annot['responsetype-vs-source']
  if('responsetype-vs-previous' in annot.keys()):
    stance[annot['tweetid']]=annot['responsetype-vs-previous']

In [166]:
stance_dict={'agreed':0,'comment':0,'appeal-for-more-information':0,'disagreed':0}
labels={'charliehebdo':copy.deepcopy(stance_dict),'sydneysiege':copy.deepcopy(stance_dict),\
        'germanwings-crash':copy.deepcopy(stance_dict),'ferguson':copy.deepcopy(stance_dict),\
        'ottawashooting':copy.deepcopy(stance_dict)}

for k,v in stance.items():
    for e in event_labels:
        if(k in replies[e]):
            labels[e][v]+=1
            break    

In [167]:
labels

{'charliehebdo': {'agreed': 214,
  'comment': 666,
  'appeal-for-more-information': 50,
  'disagreed': 72},
 'sydneysiege': {'agreed': 174,
  'comment': 677,
  'appeal-for-more-information': 87,
  'disagreed': 115},
 'germanwings-crash': {'agreed': 51,
  'comment': 163,
  'appeal-for-more-information': 33,
  'disagreed': 14},
 'ferguson': {'agreed': 145,
  'comment': 704,
  'appeal-for-more-information': 85,
  'disagreed': 108},
 'ottawashooting': {'agreed': 127,
  'comment': 450,
  'appeal-for-more-information': 65,
  'disagreed': 82}}

In [25]:
for stance in list(stance_dict.keys()):
    if(stance not in reactions):
        print('not found {}'.format(stance))

In [144]:
len(stance)

4082